In [11]:
# Constant variables definitions
DIR_PATH = "sample_texts/"


In [12]:
import os

In [13]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
import nltk
import nltk.data
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [1]:
import re

In [4]:
# split_into_sentences()
# Parameters: string of text
# Description: The function splits text into sentences while properly retaining proper punctuation
# (Note: This function was taken from StackOverflow (https://stackoverflow.com/questions/4576077/python-split-text-on-sentences)

# Handle cases where periods are used but don't note the end of a sentence
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [24]:
# 
# Parameters:
# Description: to read a text file and split it up into sentences
def process_file(fname):
    
    # Open text file
    file_path = DIR_PATH + fname
    file = open(file_path)
    
    # Open text file and connect all lines
    content = ""
    with open(DIR_PATH + fname, "r+") as file:
        content = file.readlines()
    
    content = "".join(content)
    
    # Convert the text into sentences by splitting at common symbolic stops.
    #sentence_stops = ['.','!','?',':']
    #for stop in sentence_stops:
        #content.split(stop)
    #[sentences for sentences in map(str.strip, content.split('.')) if sentences]
    #sentences = content
    #print(sentences)
    
    sentences = map(str.strip, content.split('.')) 
    
    # Use VADER to run sentiment analysis
    analyzer = SentimentIntensityAnalyzer()
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        print("{:-<65} {}".format(sentence, str(vs)))

In [26]:
def main():
    
    #process_file("sample1.txt")
    process_file("sample2.txt")
    
main()

Adaptations have long been a thorn in the side of anime viewers, but not because they are inherently bad {'neg': 0.218, 'compound': -0.6956, 'pos': 0.0, 'neu': 0.782}
No, the main problem has been that many studios have regarded the original work almost as an afterthought, and there are a number of shows that could have been wonderful if the writers had simply stuck to the original story {'neg': 0.142, 'compound': 0.4019, 'pos': 0.198, 'neu': 0.66}
One of the issues at hand seems to be ownership as producers, writers and directors all seem to want the work to be reflective of their style and perception, and in order to stamp their mark on a show they will makes numerous unnecessary changes or additions {'neg': 0.0, 'compound': 0.5423, 'pos': 0.093, 'neu': 0.907}
Admittedly there are times when the adaptation supersedes the original work, but more often than not the result is at best a decent anime, and at worst utter twaddle {'neg': 0.148, 'compound': 0.2023, 'pos': 0.196, 'neu': 0.656

In [50]:
# Below are the following files that will be tested.

files = os.listdir(DIR_PATH) # see the files in the current directory
files


['sample1.txt', 'sample2.txt', 'sample3.txt']

In [51]:
# nltk.download('stopwords')
# nltk.download('punky')

In [52]:
eng_stops = nltk.corpus.stopwords.words('english')
symbolic_stops = ['.', ',', '!', '?', ';', ':', '[', ']']

eng_stops += symbolic_stops

# TODO: 
# Implement stop words reduction.


In [45]:

# sample_index = 0
# file_chosen = files[sample_index]

# words = process_file(file_chosen)

# nltk.word_tokenize(words)


# nltk.stem.snowball.SnowballStemmer("english")




In [4]:
sentences = [ "VADER is smart, handsome, and funny.",  # positive sentence example
              "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
              "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
              "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
              "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
              "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
              "VADER is not smart, handsome, nor funny.",  # negation sentence example
              "The book was good.",  # positive sentence
              "At least it isn't a horrible book.",  # negated negative sentence with contraction
              "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
              "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
              "Today SUX!",  # negative slang with capitalization emphasis
              "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
              "Make sure you :) or :D today!",  # emoticons handled
              "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
              "Not bad at all"  # Capitalized negation
            ]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.----------------------------- {'pos': 0.746, 'compound': 0.8316, 'neu': 0.254, 'neg': 0.0}
VADER is smart, handsome, and funny!----------------------------- {'pos': 0.752, 'compound': 0.8439, 'neu': 0.248, 'neg': 0.0}
VADER is very smart, handsome, and funny.------------------------ {'pos': 0.701, 'compound': 0.8545, 'neu': 0.299, 'neg': 0.0}
VADER is VERY SMART, handsome, and FUNNY.------------------------ {'pos': 0.754, 'compound': 0.9227, 'neu': 0.246, 'neg': 0.0}
VADER is VERY SMART, handsome, and FUNNY!!!---------------------- {'pos': 0.767, 'compound': 0.9342, 'neu': 0.233, 'neg': 0.0}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!--------- {'pos': 0.706, 'compound': 0.9469, 'neu': 0.294, 'neg': 0.0}
VADER is not smart, handsome, nor funny.------------------------- {'pos': 0.0, 'compound': -0.7424, 'neu': 0.354, 'neg': 0.646}
The book was good.----------------------------------------------- {'pos': 0.492, 'compound': 0.4404, 'neu': 0.